# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Load dataset
df = pd.read_csv("dataset_clustered.csv")

In [3]:
df.sample(5)

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,TransactionType,Channel,CustomerOccupation,Cluster_Final
1827,126.91,22.0,42.0,1.0,692.69,Debit,ATM,Student,1
892,523.02,53.0,83.0,1.0,4586.11,Debit,Online,Retired,0
2348,432.18,18.0,162.0,1.0,678.83,Debit,Branch,Student,1
564,365.70,21.0,115.0,2.0,368.78,Debit,Branch,Student,1
1936,571.93,52.0,119.0,1.0,4026.03,Debit,Online,Retired,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [4]:
# Pisahkan fitur dan target
X = df.drop(columns=['Cluster_Final']) # fitur kecuali target
y = df['Cluster_Final']

# Pisahkan kolom kategorikal
categorical_cols = ['TransactionType', 'Channel', 'CustomerOccupation']
numerical_cols = X.columns.difference(categorical_cols)

# One-hot encode kategorikal, karena tadi di-inverse
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_encoded_cat = pd.DataFrame(encoder.fit_transform(X[categorical_cols]),
                             columns=encoder.get_feature_names_out(categorical_cols))

# Gabungkan numerikal dan kategorikal (sudah di encode)
X_full = pd.concat([X[numerical_cols].reset_index(drop=True),
                    X_encoded_cat.reset_index(drop=True)], axis=1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.3, random_state=42) # Data test 30%

# Scaling HANYA fitur numerik (yang awalnya numeric)
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [5]:
# Mengimplementasikan 2 algoritma klasifikasi yang berbeda untuk membandingkan performa model.
# Latih model
# Model 1 - Logistic Regression
logreg = LogisticRegression(max_iter=500, random_state=42)
logreg.fit(X_train, y_train)

# Model 2 - Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

**1. Logistic Regression**
Logistic Regression memberikan interpretasi yang jelas dalam memprediksi keanggotaan cluster.
- Algoritma ini sederhana dan interpretatif, mudah dipahami untuk menjelaskan peluang sebuah data masuk ke klaster tertentu.
- Cocok untuk kasus multi-class classification seperti prediksi klaster hasil KMeans.
- Logistic Regression bekerja optimal pada data yang sudah dinormalisasi, yang sudah sesuai dengan preprocessing dataset ini.
- Cocok juga untuk menguji seberapa linear hubungan antara fitur transaksi dan segmentasi cluster yang terbentuk.


**2. Random Forest Classifier**
Pada project ini, Random Forest dipilih karena kemampuannya dalam menangani dataset dengan kombinasi fitur numerik dan kategorikal secara efektif dengan memberi insight feature importance, serta memberi ketahanan terhadap outlier.
- Dataset ini memiliki kombinasi fitur numerik dan kategorikal (contoh: TransactionAmount, CustomerAge, TransactionType, Channel).
- Random Forest tahan terhadap outlier dan data yang tidak seimbang, cocok untuk dataset transaksi yang rentan mengandung anomali atau outlier akibat potensi fraud.
- Selain itu, Random Forest mampu memberikan feature importance, membantu untuk memahami fitur apa saja yang paling berpengaruh dalam menentukan klaster yang berisiko (fraud) atau aman.
- Secara umum, Random Forest cocok untuk data yang cukup kompleks seperti transaksi keuangan.



## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [6]:
# 1. Prediksi Logistic Regression
y_pred_logreg_train = logreg.predict(X_train)
y_pred_logreg_test = logreg.predict(X_test)

# 2. Prediksi Random Forest
y_pred_rf_train = rf.predict(X_train)
y_pred_rf_test = rf.predict(X_test)

In [7]:
# Evaluasi Akurasi
acc_logreg = accuracy_score(y_test, y_pred_logreg_test)
acc_rf = accuracy_score(y_test, y_pred_rf_test)

print(f"=== Logistic Regression Accuracy ===\nTesting Accuracy: {acc_logreg:.4f}")
print(f"=== Random Forest Accuracy ===\nTesting Accuracy: {acc_rf:.4f}")

# Evaluasi F1-Score
print("\n=== Logistic Regression F1-Score ===")
print(f"F1-Score Training: {f1_score(y_train, y_pred_logreg_train, average='weighted'):.4f}")
print(f"F1-Score Testing: {f1_score(y_test, y_pred_logreg_test, average='weighted'):.4f}")

print("=== Random Forest F1-Score ===")
print(f"F1-Score Training: {f1_score(y_train, y_pred_rf_train, average='weighted'):.4f}")
print(f"F1-Score Testing: {f1_score(y_test, y_pred_rf_test, average='weighted'):.4f}")

# Classification Report untuk detail
print("\n=== Logistic Regression Classification Report ===")
print(classification_report(y_test, y_pred_logreg_test))

print("\n=== Random Forest Classification Report ===")
print(classification_report(y_test, y_pred_rf_test))

# Confusion Matrix
print("\n=== Logistic Regression Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_logreg_test))

print("\n=== Random Forest Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_rf_test))

=== Logistic Regression Accuracy ===
Testing Accuracy: 0.9907
=== Random Forest Accuracy ===
Testing Accuracy: 0.9881

=== Logistic Regression F1-Score ===
F1-Score Training: 0.9989
F1-Score Testing: 0.9907
=== Random Forest F1-Score ===
F1-Score Training: 1.0000
F1-Score Testing: 0.9881

=== Logistic Regression Classification Report ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       569
           1       0.98      0.98      0.98       179
           2       1.00      1.00      1.00         6

    accuracy                           0.99       754
   macro avg       0.99      0.99      0.99       754
weighted avg       0.99      0.99      0.99       754


=== Random Forest Classification Report ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       569
           1       0.98      0.97      0.97       179
           2       1.00      1.00      1.00         6

    accuracy    

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

#### Evaluasi Model Logistic Regression
- Akurasi Testing: 99.07%
- F1-Score Training: 99.89%
- F1-Score Testing: 99.07%
- Precision dan Recall:
    - Kelas 0 dan kelas 2 sangat baik (hampir sempurna).
    - Kelas 1 sedikit lebih rendah tapi masih sangat baik (98%).
- Confusion Matrix:
    - Hanya 3 salah klasifikasi dari kelas 0 ke kelas 1.
    - 4 salah prediksi dari kelas 1 ke kelas 0.
    - Kelas 2 diprediksi sempurna.

#### Evaluasi Model Random Forest
- Akurasi Testing: 98.81%
- F1-Score Training: 100%
- F1-Score Testing: 98.81%
- Precision dan Recall:
    - Kelas 0 dan kelas 2 hampir sempurna.
    - Kelas 1 sedikit menurun recall-nya ke 97%.
- Confusion Matrix:
    - 4 salah klasifikasi dari kelas 0 ke kelas 1.
    - 5 salah prediksi dari kelas 1 ke kelas 0.
    - Kelas 2 diprediksi sempurna.

#### Kesimpulan
- Logistic Regression sedikit lebih unggul dalam akurasi dan f1-score testing dibanding Random Forest.
- Random Forest menunjukkan overfitting karena F1-Score Training sempurna (1.0), sementara testing turun.
- Logistic Regression lebih direkomendasikan untuk dataset ini karena lebih stabil dan tidak overfitting.
- Random Forest tetap bagus tapi perlu di-tuning lebih lanjut untuk mengurangi overfitting.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [8]:
from sklearn.model_selection import GridSearchCV
# Lakukan tuning model klasifikasi Random Forest, yang terdeteksi overfitting
# Coba parameter
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [5, 10, 20],
    'min_samples_leaf': [2, 4, 6, 8],
    'max_features': ['sqrt', 'log2']
}

# Random Forest Classifier
rf = RandomForestClassifier(random_state=42)
# GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit ke data training
grid_search.fit(X_train, y_train)

# Hasil terbaik
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy Score:", grid_search.best_score_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best Accuracy Score: 0.9863523051023051


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [9]:
best_params = grid_search.best_params_
# Buat ulang RF
rf_tuned = RandomForestClassifier(**best_params, random_state=42)
rf_tuned.fit(X_train, y_train)

y_pred_rf_train = rf_tuned.predict(X_train)
y_pred_rf_test = rf_tuned.predict(X_test)

In [10]:
y_pred_rf_train = rf_tuned.predict(X_train)
y_pred_rf_test = rf_tuned.predict(X_test)

print("Training Accuracy:", accuracy_score(y_train, y_pred_rf_train))
print("Testing Accuracy:", accuracy_score(y_test, y_pred_rf_test))

print("F1-Score Training:", f1_score(y_train, y_pred_rf_train, average='weighted'))
print("F1-Score Testing:", f1_score(y_test, y_pred_rf_test, average='weighted'))

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred_rf_test))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_rf_test))

Training Accuracy: 0.9982935153583617
Testing Accuracy: 0.986737400530504
F1-Score Training: 0.9982286397253537
F1-Score Testing: 0.986737400530504

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       569
           1       0.97      0.97      0.97       179
           2       1.00      1.00      1.00         6

    accuracy                           0.99       754
   macro avg       0.99      0.99      0.99       754
weighted avg       0.99      0.99      0.99       754


=== Confusion Matrix ===
[[564   5   0]
 [  5 174   0]
 [  0   0   6]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

#### **Hasil model Logistic Regression**
Metrik	Nilai

| **Class** | **Precision** | **Recall** | **F1-Score** | **Support** |
|----------|--------------|-----------|-------------|------------|
| **0**    | 0.99         | 0.99      | 0.99        | 569        |
| **1**    | 0.98         | 0.98      | 0.98        | 179        |
| **2**    | 1.00         | 1.00      | 1.00        | 6          |
| **Overall Accuracy** | | | **0.9907** | **754** |

- Akurasi tinggi (99.07%) baik di training maupun testing.
- Generalisasi baik → F1-Score train dan test seimbang, gap kecil → tidak overfitting.
- Semua kelas diprediksi cukup baik.

#### **Perbandingan hasil evaluasi model RandomForest sebelum dan setelah tuning**

| **Metrik**            | **Sebelum Tuning (Random Forest)** | **Sesudah Tuning (Random Forest)** |
|-----------------------|-----------------------------------|-----------------------------------|
| **Training Accuracy** | 1.0000 (100%) ✅ Indikasi Overfitting  | **0.9983 (99.8%)**  |
| **Testing Accuracy**  | 0.9881 (98.81%)                  | **0.9867 (98.67%)**
| **F1-Score Train**    | 1.0000 (100%) ✅ Indikasi Overfitting | **0.9982 (99.8%)**
| **F1-Score Test**     | 0.9881 (98.81%)                  | **0.9867 (98.67%)**
| **Best Params**       | Default                          | max_depth=10, max_features='sqrt', min_samples_leaf=2, min_samples_split=5, n_estimators=100 |

**Kesimpulan perbandingan:**
- Setelah tuning, **overfitting berkurang** (train score tidak 100%).
- **Testing performance tetap stabil** dan masih sangat baik (hampir 99%).
- Model menjadi lebih **generalizable**, bukan sekadar menghafal data train.
---
#### **Identifikasi Kelemahan Model**
#### **Logistic Regression**
- Kelas 1 tetap punya F1-score lebih rendah dari kelas 0 dan 2 → ini wajar karena support lebih kecil.
- Kelas 2 data sangat sedikit (6 data) → tetap rawan fluktuasi performa jika data bertambah.

**Kelas dengan precision atau recall rendah**
- Secara umum, tidak ada kelemahan signifikan karena precision dan recall di atas 0.95 semua.
- Namun, Kelas 1 relatif lebih rendah dibanding kelas lainnya. Ini wajar, karena biasanya kelas menengah (tidak dominan & tidak minoritas ekstrem) sering jadi yang paling susah ditebak.

#### **Random Forest**
**Kelas dengan precision atau recall rendah**
Dari classification report setelah tuning:
| Class | Precision | Recall | F1-Score | Support |
|------|----------|--------|---------|--------|
| **0** | 0.99 | 0.99 | 0.99 | 569 |
| **1** | 0.97 | 0.97 | 0.97 | 179 |
| **2** | 1.00 | 1.00 | 1.00 | 6 |

**Temuan:**  
- Kelas 1 (minor class) **masih memiliki precision dan recall lebih rendah** dibanding kelas lain (97%).
- Kelas 2 jumlahnya sangat sedikit (**hanya 6 data**) → ini berpotensi **tidak stabil** kalau data bertambah.

### **Overfitting / Underfitting**
- **Sebelum tuning:** Model cenderung **overfit** → train score sempurna (100%), test lebih rendah.
- **Sesudah tuning:** Overfit **berkurang** → gap train-test kecil, tapi train score masih tinggi.
- **Underfitting tidak terdeteksi**, karena performa tinggi di kedua dataset.
---
#### **Rekomendasi Tindakan Lanjutan**
- **Kumpulkan Data Tambahan**
    - Tambah data untuk **kelas minor (kelas 2)** → hanya 6 data sangat rawan error dan bias.
    - Pastikan distribusi label lebih **seimbang**.

- **Coba Algoritma Alternatif**
    - Tes model **Gradient Boosting**, **XGBoost**, atau **LightGBM** yang lebih kuat di data tidak seimbang.
    - Atau melakukan pengecekan **SVM dengan class_weight='balanced'** untuk melihat pengaruhnya ke kelas 1 dan 2.

- **Lakukan Cross-Validation lebih banyak**
    - Sudah pakai CV=5, kalau dataset cukup besar, bisa naik ke **CV=10** untuk hasil lebih stabil.

- **Feature Engineering / Seleksi Fitur**
    - Cek kembali fitur yang dipakai, mungkin masih ada fitur yang bisa **meningkatkan prediksi untuk kelas minor**.

---

## **Kesimpulan Akhir**
- **Model hasil tuning lebih baik dan sehat**, mengurangi overfitting.  
- **Akurasi dan F1-Score tetap tinggi** di atas 98%.  
- **Isu utama:** Kelas minor masih rawan error → perlu perhatian lebih.  
- **Next step:** Tambah data, uji algoritma lain, fokus ke penanganan imbalance.